In [4]:
from sklearn import svm, metrics
from skimage.feature import local_binary_pattern
import numpy as np
import cv2
from os import listdir
from os.path import isfile, join
from google.colab import drive
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications.xception import preprocess_input


# Montar Google Drive
drive.mount('/content/drive')

# Definir la ruta de la carpeta con las imágenes
path = '/content/drive/MyDrive/Proyecto VAI-AAI/chest_xray'

# Categorías
categorias = ['NORMAL', 'PNEUMONIA']

size = (299, 299)  # Tamaño de entrada esperado por Xception

# Cargar la base del modelo Xception con pesos preentrenados de ImageNet
base_model = Xception(weights='imagenet', include_top=False, input_shape=(size[0], size[1], 3))

# Congelar las capas del modelo base
for layer in base_model.layers:
    layer.trainable = False

# Añadir nuevas capas personalizadas
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(len(categorias), activation='softmax')(x)  # Ajustado al número de categorías

# Definir el modelo completo
model = Model(inputs=base_model.input, outputs=predictions)

# Compilar el modelo
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])


def cargar_datos(ruta, size=(299, 299)):
    datos = []
    etiquetas = []
    for i, cat in enumerate(categorias):
        carpeta = join(ruta, cat)
        archivos = [f for f in listdir(carpeta) if isfile(join(carpeta, f))]
        for archivo in archivos:
            imagen_path = join(carpeta, archivo)
            imagen = cv2.imread(imagen_path)
            if imagen is None:
                print(f'No se pudo leer la imagen: {imagen_path}')
                continue

            # Preparar la imagen para Xception
            imagen = cv2.resize(imagen, size)
            imagen = cv2.cvtColor(imagen, cv2.COLOR_BGR2RGB)
            imagen = img_to_array(imagen)
            imagen = np.expand_dims(imagen, axis=0)
            imagen = preprocess_input(imagen)

            # Usar Xception para extraer características
            caracteristicas = base_model.predict(imagen)
            caracteristicas = caracteristicas.flatten()

            datos.append(caracteristicas)
            etiquetas.append(i)

    if len(datos) == 0 or len(etiquetas) == 0:
        raise ValueError("No se pudieron cargar los datos. Asegúrate de que la ruta y las categorías son correctas.")

    return np.array(datos), np.array(etiquetas)

# Cargar datos de entrenamiento, validación y prueba
datos_entrenamiento, etiquetas_entrenamiento = cargar_datos(join(path, 'train'))
datos_prueba, etiquetas_prueba = cargar_datos(join(path, 'test'))

# Asegurarse de que hay datos para al menos dos clases
if len(np.unique(etiquetas_entrenamiento)) < 2 or len(np.unique(etiquetas_prueba)) < 2:
    raise ValueError("Se debe cargar datos para al menos dos clases en cada conjunto de datos.")

# Crear y entrenar el modelo SVM
modelo = svm.SVC(gamma='scale')
modelo.fit(datos_entrenamiento, etiquetas_entrenamiento)

# Evaluar el modelo
etiquetas_predichas = modelo.predict(datos_prueba)
precision = metrics.accuracy_score(etiquetas_prueba, etiquetas_predichas) * 100
print(f'Precisión del modelo: {precision:.2f}%')

Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 384ms/step
